Incercam folosirea unui model putin mai mare. Problema acestui llm este faptul ca nu este antrenat special pentru limba Romana

In [ ]:
!pip install -q transformers accelerate torch sentencepiece


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map={"": "cpu"}
)

print("Model încărcat pe CPU:", MODEL_NAME)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model încărcat pe CPU: TinyLlama/TinyLlama-1.1B-Chat-v1.0


Urmam modelul de la Fabule. De data aceasta putem da un SYSTEM_MSG

In [ ]:
SYSTEM_MSG = (
    "Ești un jurist care redactează cereri judiciare clare, concise şi conforme codului de procedură civilă din România. "
    "Creează text complet în limba română, folosind formule standard juridice precum „Subsemnatul…”, „Pentru aceste motive…”, „Vă rugăm să…”."
)

TEMPLATE = """
Redactează o {tip_cerere} completă în limba română, pe baza următoarelor date:
- Instanță: {instanta}
- Reclamant / Apelant / Recursant: {reclamant}
- Pârât / Intimat: {parat}
- Obiectul cererii: {obiect}
- Temei legal: {temei}
- Probe: {probe}
- Ton: formal și juridic
- Lungime dorită: {lungime} cuvinte
"""

tipuri = ["cerere de chemare în judecată", "cerere de apel", "cerere de recurs"]
instante = ["Judecătoria București", "Tribunalul Cluj", "Curtea de Apel Iași"]
reclamanti = ["subsemnatul A.B.", "apelantul C.D.", "recursantul E.F."]
parati = ["Societatea X S.R.L.", "pârâtul G.H.", "intimatul I.J."]
obiecte = [
    "obligarea pârâtului la plata sumei de 10.000 lei reprezentând daune morale",
    "anularea hotărârii civile nr. 123/2024 a Tribunalului Cluj",
    "modificarea dispozitivului sentinței și respingerea acțiunii ca neîntemeiată"
]
temeiuri = [
    "art. 1349 Cod civil și art. 194 C.proc.civ.",
    "art. 466–480 C.proc.civ.",
    "art. 488 alin. 1 pct. 8 C.proc.civ."
]
probe = [
    "înscrisuri, martori, înregistrări video",
    "contractul anexat, corespondența electronică, facturi",
    "hotărâri anterioare și dovezi de plată"
]
lungimi = [150, 200, 250]

def sample_prompt():
    return TEMPLATE.format(
        tip_cerere=random.choice(tipuri),
        instanta=random.choice(instante),
        reclamant=random.choice(reclamanti),
        parat=random.choice(parati),
        obiect=random.choice(obiecte),
        temei=random.choice(temeiuri),
        probe=random.choice(probe),
        lungime=random.choice(lungimi)
    )

In [ ]:
def call_model(system, prompt, temperature=0.3, max_new_tokens=400):
    full_prompt = f"System: {system}\nUser: {prompt}\nAssistant:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to("cpu")  # CPU
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
import random
p = sample_prompt()
print(f"\n=== Prompt ===\n{p}")
out = call_model(SYSTEM_MSG, p)
print(f"\n--- Cerere generată ---\n{out}\n")



=== Prompt ===

Redactează o cerere de apel completă în limba română, pe baza următoarelor date:
- Instanță: Tribunalul Cluj
- Reclamant / Apelant / Recursant: recursantul E.F.
- Pârât / Intimat: Societatea X S.R.L.
- Obiectul cererii: obligarea pârâtului la plata sumei de 10.000 lei reprezentând daune morale
- Temei legal: art. 466–480 C.proc.civ.
- Probe: contractul anexat, corespondența electronică, facturi
- Ton: formal și juridic
- Lungime dorită: 200 cuvinte


--- Cerere generată ---
System: Ești un jurist care redactează cereri judiciare clare, concise şi conforme codului de procedură civilă din România. Creează text complet în limba română, folosind formule standard juridice precum „Subsemnatul…”, „Pentru aceste motive…”, „Vă rugăm să…”.
User: 
Redactează o cerere de apel completă în limba română, pe baza următoarelor date:
- Instanță: Tribunalul Cluj
- Reclamant / Apelant / Recursant: recursantul E.F.
- Pârât / Intimat: Societatea X S.R.L.
- Obiectul cererii: obligarea pârâtu

Incercam rularea modelului in limba Engleza, pentru a vedea daca rezulatatele obtinute sunt putin mai bune

In [ ]:

!pip install -q transformers accelerate torch sentencepiece pandas

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random
import pandas as pd

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.float32,
    device_map={"": "cpu"}
)

print("Model încărcat pe CPU:", MODEL_NAME)
print("GPU disponibil:", torch.cuda.is_available())


SYSTEM_MSG = (
    "You are a lawyer assistant who writes clear, concise, and legally correct court petitions "
    "according to the Romanian Civil Procedure Code. "
    "All petitions must be written in Romanian, using standard legal formulas such as “Subsemnatul…”, “Pentru aceste motive…”, “Vă rugăm să…”."
)


TEMPLATE = """
Generate a complete {petition_type} in Romanian based on the following information:
- Court: {court}
- Claimant / Appellant / Plaintiff: {claimant}
- Defendant / Respondent: {defendant}
- Object of the petition: {object}
- Legal basis: {legal_basis}
- Evidence: {evidence}
- Tone: formal and legal
- Desired length: {length} words
"""


petition_types = ["lawsuit petition", "appeal petition", "appeal in cassation"]
courts = ["Bucharest Court", "Cluj Tribunal", "Iași Court of Appeal"]
claimants = ["the undersigned A.B.", "appellant C.D.", "appellant E.F."]
defendants = ["Company X S.R.L.", "defendant G.H.", "respondent I.J."]
objects = [
    "obliging the defendant to pay 10,000 lei as moral damages",
    "annulment of civil decision no. 123/2024 of the Cluj Tribunal",
    "modification of the sentence and rejection of the action as unfounded"
]
legal_bases = [
    "Art. 1349 Civil Code and Art. 194 Romanian Civil Procedure Code",
    "Arts. 466–480 Romanian Civil Procedure Code",
    "Art. 488 para. 1 point 8 Romanian Civil Procedure Code"
]
evidence_list = [
    "documents, witnesses, video recordings",
    "attached contract, email correspondence, invoices",
    "previous decisions and proof of payment"
]
lengths = [150, 200, 250]


def sample_prompt():
    return TEMPLATE.format(
        petition_type=random.choice(petition_types),
        court=random.choice(courts),
        claimant=random.choice(claimants),
        defendant=random.choice(defendants),
        object=random.choice(objects),
        legal_basis=random.choice(legal_bases),
        evidence=random.choice(evidence_list),
        length=random.choice(lengths)
    )

def call_model(system, prompt, temperature=0.3, max_new_tokens=400):
    full_prompt = f"System: {system}\nUser: {prompt}\nAssistant:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to("cpu")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


p = sample_prompt()
print(f"\n=== Prompt ===\n{p}")
out = call_model(SYSTEM_MSG, p)
print(f"\n--- Cerere generată ---\n{out}\n")



✅ Model încărcat pe CPU: TinyLlama/TinyLlama-1.1B-Chat-v1.0
GPU disponibil: False

=== Prompt ===

Generate a complete appeal in cassation in Romanian based on the following information:
- Court: Iași Court of Appeal
- Claimant / Appellant / Plaintiff: appellant E.F.
- Defendant / Respondent: respondent I.J.
- Object of the petition: annulment of civil decision no. 123/2024 of the Cluj Tribunal
- Legal basis: Arts. 466–480 Romanian Civil Procedure Code
- Evidence: attached contract, email correspondence, invoices
- Tone: formal and legal
- Desired length: 150 words


--- Cerere generată ---
System: You are a lawyer assistant who writes clear, concise, and legally correct court petitions according to the Romanian Civil Procedure Code. All petitions must be written in Romanian, using standard legal formulas such as “Subsemnatul…”, “Pentru aceste motive…”, “Vă rugăm să…”.
User: 
Generate a complete appeal in cassation in Romanian based on the following information:
- Court: Iași Court of 